In [1]:
import bayes3d as j
import jax
import jax.numpy as jnp
import numpy as np
import os
import trimesh
import copy
import open3d as o3d
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import io

intrinsics = j.Intrinsics(
    height=100,
    width=100,
    fx=100.0, fy=100.0,
    cx=50.0, cy=50.0,
    near=1.0, far=500.0
)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
j.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [3]:
table_params = (1000.0,
    1000.0,
    0.01,
    0.01,
    0.01)
table_mesh = j.mesh.make_table_mesh(
    *table_params
)
table_dims = j.utils.aabb(table_mesh.vertices)[0]
table_pose = j.t3d.inverse_pose(
    j.t3d.transform_from_pos_target_up(
        jnp.array([0.0, 300.0, 150.0]),
        jnp.array([0.0, 0.0, 0.0]),
        jnp.array([0.0, 0.0, 1.0]),
    )
)
contact_plane = table_pose @ j.scene_graph.get_contact_planes(table_dims)[2]

In [4]:
occluder = j.mesh.make_cuboid_mesh(jnp.array([10.0, 70.0, 100.0]))                       
obj_a = j.mesh.make_cuboid_mesh(jnp.array([50.0, 100.0, 50.0]))                       
obj_b = j.mesh.make_cuboid_mesh(jnp.array([50.0, 60.0, 50.0]))
obj_c = j.mesh.make_cuboid_mesh(jnp.array([50.0, 20.0, 50.0]))
renderer = j.Renderer(intrinsics, num_layers=1024)
renderer.add_mesh(occluder)
renderer.add_mesh(obj_a)
renderer.add_mesh(obj_b)
renderer.add_mesh(obj_c)

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (128, 128, 1024)


In [5]:
def sample_image(object_poses, object_ids, r_array):
    reconstruction = renderer.render_multiobject(
        object_poses, object_ids
    )
    rendered_seg = reconstruction[:,:,3]
    r = r_array[jnp.abs(rendered_seg[..., None] - jnp.arange(len(r_array))).argmin(-1)]
    key = jax.random.PRNGKey(10)
    noisy_point_cloud_image = jax.random.multivariate_normal(
        key, reconstruction[:,:,:3], (jnp.eye(3)[None, None, :, :] * r[:,:,None,None]), shape=r.shape
    )
    img = j.render_point_cloud(noisy_point_cloud_image.reshape(-1,3), intrinsics)
    return img

def inference():
    all_hypotheses = []
    for obj_id in range(1,len(renderer.meshes)):
#         print("Object ID ", obj_id)
        contact_param = jnp.zeros(3)
        p = None
        traces = None
        for c2f_iter in range(len(contact_param_gridding_schedule)):
            contact_param_grid = contact_param_gridding_schedule[c2f_iter] + contact_param

            potential_new_object_poses = contact_plane @ pose_from_contact_and_face_params_parallel_jit(
                contact_param_grid,
                3,
                renderer.model_box_dims[obj_id],
            )
            POTENTIAL_R = jnp.hstack([jnp.tile(R[None,:], (R_SWEEP.shape[0], 1)),R_SWEEP])

            potential_poses = jnp.concatenate(
                [
                    jnp.tile(object_poses, (1,potential_new_object_poses.shape[0],1,1)),
                    potential_new_object_poses[None,...]
                ]
            )
            
            traces = j.Traces(
                potential_poses, object_ids + [obj_id], POTENTIAL_R, OUTLIER_PROBS, OUTLIER_VOLUME, observed_point_cloud_image
            )
            p = j.score_traces(traces, renderer)
            ii,jj,kk = jnp.unravel_index(p.argmax(), p.shape)
            contact_param = contact_param_grid[ii]

        all_hypotheses.append(
            traces[ii,jj,kk]
        )

    return all_hypotheses

In [6]:
sched = [(100.0, jnp.pi), (50.0, jnp.pi), (10.0, jnp.pi),
         (5.0, jnp.pi/2),(5.0, jnp.pi/2),(5.0, jnp.pi/2),
         (2.0, jnp.pi/2),(2.0, jnp.pi/2),(2.0, jnp.pi/2),
         (1.0, jnp.pi/4),(1.0, jnp.pi/4),(1.0, jnp.pi/4),
]

contact_param_gridding_schedule = [
    j.make_translation_grid_enumeration_3d(
        -x, 0.0, -y,
        x, 0.0, y,
        13,1,3
    )
    for (x,y) in sched
]

pose_from_contact_and_face_params_parallel_jit = jax.jit(jax.vmap(
    j.scene_graph.relative_pose_from_edge, in_axes=(0,None, None))
)

R_SWEEP = jnp.array([100.0, 50.0, 10.0, 5.0]).reshape(-1,1)
print(R_SWEEP)
OUTLIER_PROBS = jnp.linspace(0.01, 0.2, 4)
OUTLIER_VOLUME = 100.0**3 * 0.01
SCALING_FACTOR = 3

[[100.]
 [ 50.]
 [ 10.]
 [  5.]]


In [8]:
object_indices = jnp.array([0,1])

poses = contact_plane @ j.scene_graph.relative_pose_from_edge_parallel_jit(
    jnp.array(
        [
            [0.0, 100.0, 0.0],
            [-60.0, 0.0, 0.0],
        ]
    ),
    jnp.array([3,3]),
    renderer.model_box_dims[object_indices],
)
SCALING_FACTOR = 4
observed_point_cloud_image = renderer.render_multiobject(poses, object_indices)
j.show_cloud("obs", observed_point_cloud_image[:,:,:3].reshape(-1,3) / 1000.0)
j.scale_image(j.get_depth_image(observed_point_cloud_image[:,:,2]),SCALING_FACTOR)

R = jnp.array([10.0, 10.0])
object_poses, object_ids = poses[:1,:,:][None,...], [object_indices[0]]
reconstruction = renderer.render_multiobject(
    object_poses, object_ids
)

all_hypotheses = inference()

scores = jnp.array([j.score_trace(t, renderer) for t in all_hypotheses])
normalized_scores = j.utils.normalize_log_scores(scores)
print("SCORES  ", scores)
print("NORMALIZED SCORES  ", normalized_scores)
print(jnp.argsort(-scores))
best_hypothesis = all_hypotheses[jnp.argmax(scores)]
print("Best")
print(best_hypothesis)

R = best_r
object_poses = best_poses[:,None,...]
object_ids = object_ids + [obj_id]
print(object_ids)



# height_factor = outlier_prob_viz.height / observed_image_viz.height

plt.clf()
plt.figure(figsize=(7,4))
color = np.array([229, 107, 111])/255.0
plt.bar(
    np.arange(len(normalized_scores)),
    normalized_scores,
    color=color
)
plt.xticks(np.arange(len(normalized_scores)), ["{:d}".format(i) for i in range(len(normalized_scores))],fontsize=15)
plt.yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0],fontsize=15)
plt.xlabel("Object ID",fontsize=20)
plt.ylabel("Probability",fontsize=20)
plt.ylim(0.0, 1.05)
plt.tight_layout()
img_buf = io.BytesIO()
plt.savefig(img_buf, format='png')
posterior_bar_chart = Image.open(img_buf)

depth_viz = j.scale_image(j.get_depth_image(observed_point_cloud_image[:,:,2]), SCALING_FACTOR)

height_factor = depth_viz.height / posterior_bar_chart.height


viz1 = j.multi_panel(
    [
        depth_viz,
        j.scale_image(j.get_depth_image(reconstruction[:,:,2]), SCALING_FACTOR),
        j.scale_image(posterior_bar_chart, height_factor)
    ],
    labels=[
        "Observed Image",
        "Reconstruction",
        "Posterior on Object Type",
    ],
    middle_width=30
)

viz2 = j.multi_panel(
    [j.scale_image(j.get_depth_image(r[1][:,:,2]), SCALING_FACTOR) for r in all_hypotheses],
    labels=["obj_{:d}".format(idx, i.item()) for (idx,i) in enumerate(normalized_scores)],
    title="Inferred Pose per Object Type"
)

viz3 = j.multi_panel(
    [j.scale_image(j.get_depth_image(sample_image(r[4][None,...], [object_indices[0],r[5]], r[2])[:,:,2]), SCALING_FACTOR) for r in all_hypotheses],
    labels=["obj_{:d}, Noise: {:0.2f}".format(idx, i[2][-1].item()) for (idx,i) in enumerate(all_hypotheses)],
    title="Posterior Samples per Object Type"
)

final_viz = j.vstack_images([viz1, viz2])
final_viz

SCORES   [-107797.39 -107797.42 -107810.22]
NORMALIZED SCORES   [5.0677532e-01 4.9118349e-01 1.3602903e-06]
[0 1 2]
Best
variance: [10. 10.  5.] outlier_prob: 0.20000000298023224 outlier_volume: 10000.0
 ids: [Array(0, dtype=int32), 1] poses: [[[ 5.9604645e-08  1.0000000e+00  2.5611371e-08  1.2805685e-06]
  [ 4.4726557e-01  4.9569280e-08 -8.9453125e-01 -4.4784546e-03]
  [-8.9453125e-01 -4.1863132e-08 -4.4726557e-01  2.2368359e+02]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]

 [[-8.3469786e-08 -1.0000000e+00 -1.4924444e-07  6.1156246e+01]
  [-4.4726557e-01  1.1750575e-07 -8.9453125e-01 -2.2367760e+01]
  [ 8.9453125e-01  9.8748039e-08 -4.4726557e-01  3.2431613e+02]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]]


TypeError: __str__ returned non-string (type NoneType)

In [ ]:
viz_images = []
for block_x in jnp.linspace(-120.0, 120.0, 50):
    print("x :", block_x)
    object_indices = jnp.array([0,1])

    poses = contact_plane @ j.scene_graph.relative_pose_from_edge_parallel_jit(
        jnp.array(
            [
                [0.0, 100.0, 0.0],
                [block_x, 0.0, 0.0],
            ]
        ),
        jnp.array([3,3]),
        renderer.model_box_dims[object_indices],
    )
    SCALING_FACTOR = 4
    observed_point_cloud_image = renderer.render_multiobject(poses, object_indices)
    j.scale_image(j.get_depth_image(observed_point_cloud_image[:,:,2]),SCALING_FACTOR)


    R = jnp.array([10.0, 1.0])
    object_poses, object_ids = poses[:1,:,:][None,...], [object_indices[0]]
    reconstruction = renderer.render_multiobject(
        object_poses, object_ids
    )

    all_hypotheses = inference()


    scores = jnp.array([i[0] for i in all_hypotheses])
    normalized_scores = j.utils.normalize_log_scores(scores)
    print("SCORES  ", scores)
    print("NORMALIZED SCORES  ", normalized_scores)
    print(jnp.argsort(-scores))
    best_hypothesis = all_hypotheses[jnp.argmax(scores)]
    score, reconstruction, best_r, best_outlier_prob, best_poses, obj_id = best_hypothesis
    print("Best")
    print(best_r, best_outlier_prob, obj_id)

    R = best_r
    object_poses = best_poses[:,None,...]
    object_ids = object_ids + [obj_id]
    print(object_ids)

    # outliers = (
    #     (1.0 - best_outlier_prob) * (j.gaussian_mixture_image_multi_r_jit(observed_point_cloud_image, reconstruction[:,:,:3], reconstruction[:,:, -1], best_r))
    #         <
    #     (best_outlier_prob / OUTLIER_VOLUME)
    # )

    # height_factor = outlier_prob_viz.height / observed_image_viz.height
    plt.clf()
    plt.figure(figsize=(7,4))
    color = np.array([229, 107, 111])/255.0
    plt.bar(
        np.arange(len(normalized_scores)),
        normalized_scores,
        color=color
    )
    plt.xticks(np.arange(len(normalized_scores)), ["{:d}".format(i) for i in range(len(normalized_scores))],fontsize=15)
    plt.yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0],fontsize=15)
    plt.xlabel("Object ID",fontsize=20)
    plt.ylabel("Probability",fontsize=20)
    plt.ylim(0.0, 1.05)
    plt.tight_layout()
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png')
    posterior_bar_chart = Image.open(img_buf)

    depth_viz = j.scale_image(j.get_depth_image(observed_point_cloud_image[:,:,2]), SCALING_FACTOR)

    height_factor = depth_viz.height / posterior_bar_chart.height



    # outlier_viz = j.scale_image(j.get_depth_image(1.0 - outliers), SCALING_FACTOR)
    viz1 = j.multi_panel(
        [
            depth_viz,
            j.scale_image(j.get_depth_image(reconstruction[:,:,2]), SCALING_FACTOR),
            j.scale_image(posterior_bar_chart, height_factor)
        ],
        labels=[
            "Observed Image",
            "Reconstruction",
            "Posterior on Object Type",
        ],
        middle_width=30
    )

    viz2 = j.multi_panel(
        [j.scale_image(j.get_depth_image(r[1][:,:,2]), SCALING_FACTOR) for r in all_hypotheses],
        labels=["obj_{:d}".format(idx, i.item()) for (idx,i) in enumerate(normalized_scores)],
        title="Inferred Pose per Object Type"
    )

    final_viz = j.vstack_images([viz1, viz2])
    viz_images.append(final_viz)


In [9]:
j.make_gif(viz_images, "out.gif")